In [40]:
import numpy as np
from sklearn.datasets import make_classification, load_iris
from sklearn.model_selection import train_test_split
import torch

In [41]:
X, y = make_classification(n_samples=500, n_informative = 7, n_classes=3, n_features=10)
# X,y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [42]:
from simple_model import ClassifierModel, DyadRankingModel

clf = ClassifierModel(input_dim = X_train.shape[1], hidden_dim=16, output_dim=y.max()+1)
rnk = DyadRankingModel(input_dim = X_train.shape[1] + y.max()+1, hidden_dim=16)

clf.fit(X_train,y_train, num_epochs=1000, batch_size=1, learning_rate=0.001)
rnk.fit(X_train,y_train, num_epochs=1000, batch_size=1, learning_rate=0.001)

In [48]:
from sklearn.metrics import accuracy_score
y_pred_clf = clf.predict(X_test)
y_pred_rnk = rnk.predict_classes(X_test)


In [49]:
accuracy_score(y_pred_clf, y_test)

0.8666666666666667

In [50]:
accuracy_score(y_pred_rnk, y_test)

0.8181818181818182

In [46]:
print(y_pred_rnk)

[0 1 1 1 1 2 0 1 2 2 0 1 2 1 1 2 0 1 1 1 2 0 1 0 1 1 0 1 0 0 2 2 2 0 0 2 1
 2 2 0 1 2 2 1 2 1 0 1 2 0 0 2 2 0 0 2 2 0 0 1 2 2 2 0 1 0 1 2 1 1 0 1 0 1
 0 2 2 0 0 0 1 2 0 1 2 0 2 2 2 2 1 0 1 1 2 2 0 1 1 0 1 0 1 0 2 0 2 0 0 0 2
 1 2 2 1 2 2 0 1 2 0 1 2 1 1 0 0 2 1 2 2 2 0 1 0 1 2 1 0 0 2 1 0 1 2 1 0 0
 1 2 0 2 0 2 1 2 0 0 1 2 2 0 0 0 1]


In [47]:
X = torch.tensor(X_test, dtype=torch.float32)
eye = torch.eye(3)
eyes = eye.repeat((X.shape[0], 1))
Xs = X.repeat_interleave(3, dim=0)
dyads = torch.cat([Xs, eyes], dim=1)

In [39]:
rnk(dyads).view(-1,3)

tensor([[5.0186, 7.7470, 5.2435],
        [4.0988, 6.7357, 6.5838],
        [3.8287, 5.2760, 7.4289],
        [4.0126, 6.2560, 7.2981],
        [4.7854, 7.6715, 5.6353],
        [8.2664, 4.7492, 3.8967],
        [4.0590, 6.5551, 7.1697],
        [4.0010, 6.2745, 7.6428],
        [4.0624, 6.5341, 7.6514],
        [4.3845, 7.2767, 6.2124],
        [4.2930, 7.0929, 6.0766],
        [4.0665, 6.6173, 6.9221],
        [8.2810, 4.7097, 3.9061],
        [3.8768, 5.5905, 7.2696],
        [3.9843, 6.0960, 7.6634],
        [4.0431, 6.4472, 7.4900],
        [3.8955, 5.7183, 7.7014],
        [4.5917, 7.5845, 6.2840],
        [8.2492, 4.6969, 3.8855],
        [4.2216, 6.9865, 8.0393],
        [4.7688, 7.5516, 5.2012],
        [3.9083, 5.6178, 7.7365],
        [5.7593, 7.7523, 4.6139],
        [5.0350, 7.9346, 6.0756],
        [8.2639, 4.7287, 3.9185],
        [4.6044, 7.6353, 6.4568],
        [3.9280, 5.6605, 7.6969],
        [8.1683, 4.6304, 3.9156],
        [8.3246, 4.9968, 3.8733],
        [4.769